In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setup PYTHONPATH
import sys
sys.path += ['.', './extern/scannetpp', './extern/mast3r', './extern/mast3r/dust3r']

In [9]:
from pathlib import Path
import torch
from torchvision.io import read_image

import pyvista as pv
pv.set_jupyter_backend('server')

from dataset import SceneDataset, SceneDatasetTransformToTorch
from einops import rearrange
from models.surface_net_baseline.model import SimpleOccNetConfig
from models.surface_net_baseline.module import LRConfig, OccSurfaceNet, OptimizerConfig
from models.surface_net_baseline.util import project_points_to_images
from utils.data_parsing import load_yaml_munch
from lightning import Trainer
from lightning.pytorch.loggers import WandbLogger
import lightning as pl

from utils.visualize import visualize_mesh
from utils.basic import get_default_device


In [4]:
# Init dataset and load for visualization
dataset = SceneDataset(
    data_dir="datasets/scannetpp/data",
    camera="iphone",
    n_points=300000,
    threshold_occ=0.01,
    representation="occ",
    visualize=False,
)

device = get_default_device()
transform = SceneDatasetTransformToTorch(target_device=device)

In [6]:
idx = dataset.get_index_from_scene("8b2c0938d6")
data = dataset[idx]

image_names, camera_params_list, _ = data["images"]
images, transformations, points, gt = transform.forward(data)
# and normalize images
images = images / 255.0

/Users/luca/uni/master/dl-in-vc/voxl3r/dataset.py:301: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  points = torch.tensor(torch.from_numpy(points).float()).to(self.target_device)
/Users/luca/uni/master/dl-in-vc/voxl3r/dataset.py:302: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(torch.from_numpy(gt).float()).to(self.target_device)


In [10]:
from dataset import plot_training_example

plot_training_example(dataset, idx)

Widget(value='<iframe src="http://localhost:59748/index.html?ui=P_0x28ef8d070_2&reconnect=auto" class="pyvista…

In [ ]:

# from dataset import plot_mask

# plot_mask(dataset, idx)

In [11]:
from dataset import plot_occupency_grid

plot_occupency_grid(dataset, idx)

Voxel Grid Size (L, W, H): (np.float64(1.9772891040177676), np.float64(2.392200282379526), np.float64(2.1013366641606663))
Voxel Grid Center: [4.07553703 1.72495476 0.60615678]
Occupied Voxels: 12552.0


Widget(value='<iframe src="http://localhost:59748/index.html?ui=P_0x28ef90e00_3&reconnect=auto" class="pyvista…